In [1]:
from openfisca_uk import Microsimulation
from openfisca_uk import *
from openfisca_core.model_api import Reform
from openfisca_uk.entities import Person, BenUnit, Household

Creating the reform

In [2]:
sim = Microsimulation()

class PA_reform(Reform):
    def apply(self):
        self.neutralize_variable("personal_allowance")
        
sim_no_PA = Microsimulation(PA_reform)

In [3]:
revenue = sim.calc("household_net_income").sum()
revenue_diff = revenue - sim_no_PA.calc("household_net_income").sum()
revenue_diff

96107384023.48047

In [4]:
BI_amount = revenue_diff/(sim.calc("people").sum())
BI_amount

1467.7688599493447

In [5]:
from openfisca_core.model_api import *
from openfisca_uk.tools.general import *

class BI(Variable):
    value_type = float
    entity = Person
    label = u"UBI"
    definition_period = YEAR
    def formula(person, period, parameters):
        return(BI_amount)

class gross_income(Variable):
    value_type = float
    entity = Person
    label = u"Gross income, including benefits"
    definition_period = YEAR

    def formula(person, period, parameters):
        COMPONENTS = [
            "employment_income",
            "pension_income",
            "self_employment_income",
            "property_income",
            "savings_interest_income",
            "dividend_income",
            "miscellaneous_income",
            "benefits",
            "BI"
        ]
        return add(person, period, COMPONENTS)   

class basic_income(Reform):
    def apply(self):
        self.add_variable(BI)
        self.update_variable(gross_income)
        
sim_BI = Microsimulation(PA_reform, basic_income)

Overall effect on income & taxes

In [6]:
#Difference in median income tax
sim_BI.calc("income_tax").median() - sim.calc("income_tax").median()

1954.0020725340225

In [7]:
#Increase in median household net income
sim_BI.calc("household_net_income").median() - sim.calc("household_net_income").median()

486.6101750087

In [8]:
#Decrease in mean household net income
sim_BI.calc("household_net_income").mean() - sim.calc("household_net_income").mean()

-15.227619815676007

Effect on each income group

In [9]:
income_diff = sim_BI.calc("household_net_income", map_to="household") - sim.calc("household_net_income", map_to="household")

In [10]:
income = sim.calc("household_net_income", map_to="household")

In [11]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"

In [12]:
#Median difference in household net income for each income decile
px.bar(income_diff.groupby(income.decile_rank()).median())

In [13]:
#Mean difference in household net income for each income decile
px.bar(income_diff.groupby(income.decile_rank()).mean())

Effect on poverty

In [14]:
#Median difference in household net income for those in deep poverty vs those not in deep poverty
isDeepPoor = sim.calc("in_deep_poverty_bhc", map_to ="household")
px.bar(income_diff.groupby(isDeepPoor).median())

In [15]:
#Median difference in household net income for those in poverty vs those not in deep poverty
isPoor = sim.calc("in_poverty_bhc", map_to = "household")
px.bar(income_diff.groupby(isPoor).median())

In [16]:
#Percentage of poverty reduction
poverty_difference = sim_BI.calc("in_poverty_bhc").sum() - sim.calc("in_poverty_bhc").sum()
(poverty_difference/sim.calc("in_poverty_bhc").sum()) * 100

-16.67232192046633

In [17]:
#Percentage of deep poverty redution 
deep_poverty_difference = sim_BI.calc("in_deep_poverty_bhc").sum() - sim.calc("in_deep_poverty_bhc").sum()
(deep_poverty_difference/sim.calc("in_deep_poverty_bhc").sum()) * 100

-33.10014218076008

Effect on child poverty

In [18]:
#Percentage of child poverty reduction
poor_kids_before = sim.calc("in_poverty_bhc", map_to="person")[sim.calc("is_child")].sum()
poor_kids_after = sim_BI.calc("in_poverty_bhc", map_to="person")[sim_BI.calc("is_child")].sum()
((poor_kids_before - poor_kids_after)/poor_kids_before) * 100

48.8483209064882

In [19]:
person_income_diff = sim_BI.calc("net_income", map_to="person") - sim.calc("net_income", map_to="person")

In [20]:
#Mean difference in income by age
age = sim.calc("age", map_to ="person")
px.bar(person_income_diff.groupby(age).mean())

In [21]:
#Mean difference in income by child vs adult
is_child = sim.calc("is_child", map_to ="person")
px.bar(person_income_diff.groupby(is_child).mean())

In [22]:
#Percentage of poverty reduction for adults only
poor_adults_before = sim.calc("in_poverty_bhc", map_to="person")[sim.calc("is_adult")].sum()
poor_adults_after = sim_BI.calc("in_poverty_bhc", map_to="person")[sim_BI.calc("is_adult")].sum()
((poor_adults_before - poor_adults_after)/poor_adults_before) * 100

20.330388824845965

In [23]:
#Mean income difference grouped by family type
family = sim.calc("family_type", map_to = "person")
px.bar(person_income_diff.groupby(family).mean())

Effect on other demographics

In [24]:
gender = sim.calc("gender")
px.bar(person_income_diff.groupby(gender).median())
#only 1 col